In [9]:
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import flwr as fl
import tensorflow as tf

import pandas as pd

from flwr.common.logger import log
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)




In [10]:
inputFolders = "../data_2019_processed/"
# selected features
inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]

In [11]:

def loadTestData(dataSet):
    print(len(dataSet), "datasets")
    for i in range(0,len(dataSet)):
        print(i , "-", dataSet[i])
        #print(trainingDataSet[i])
        if(i == 0):
            X_test = pd.read_csv(inputFolders+"student_"+dataSet[i]+"_transformed.csv")
        else:
            dataset = pd.read_csv(inputFolders+"student_"+dataSet[i]+"_transformed.csv")
            X_test = pd.concat([X_test, dataset])
    # return the dataset        
    return X_test

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

In [12]:
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)

        if aggregated_parameters is not None:
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)

            # Save aggregated_ndarrays
            print(f"Saving round {server_round} aggregated_ndarrays...")
            np.savez(f"round-{server_round}-weights.npz", *aggregated_ndarrays)

        return aggregated_parameters, aggregated_metrics

In [13]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Keep batch size fixed at 32, perform two rounds of training with one
    local epoch, increase to two local epochs afterwards.
    """
    print("onn fit")
    config = {
        "batch_size": 32,
        "local_epochs": 1,
    }
    return config
	
	
def get_evaluate_fn(model):
    """Return an evaluation function for server-side evaluation."""
    
    testFolders =  [#'0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                #'0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                #'2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                #'2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                #'7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                #'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                #'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                #'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                #'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                #'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                #'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                #'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                #'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                #'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                #'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                #'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                #'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                #'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                #'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
                'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
    
    # load data
    X_test = loadTestData(testFolders)
    # transform output to one_hot_encoding for the testing dataset
    X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)
    # transforms the data
    X_test = transform_data_type(X_test)
    # selects the data to train and test
    X_test_data = X_test[inputFeatures]
    y_test_label = X_test[outputClasses]
 
    # The `evaluate` function will be called after every round
    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        model.set_weights(parameters)  # Update model with the latest parameters
        loss, accuracy = model.evaluate(X_test_data, y_test_label)
        print("Testa aquiiiiiiiiiiiiiiiiiiiiiiii", loss, accuracy)
        return loss, {"accuracy": accuracy}

    return evaluate
	
def evaluate_config(server_round: int):
    """Return evaluation configuration dict for each round.
    Perform five local evaluation steps on each client (i.e., use five
    batches) during rounds, one to three, then increase to ten local
    evaluation steps.
    """
    val_steps = 5 if server_round < 4 else 10
    return {"val_steps": val_steps}

In [14]:
def create_keras_model():
    return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(9,)),
      #tf.keras.layers.Dense(9, activation=tf.keras.activations.relu), 
      tf.keras.layers.Dense(16, activation=tf.keras.activations.relu),
      tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),
      tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
      #tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
    ])
# Load model and data (MobileNetV2, CIFAR-10)
model = create_keras_model()
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [15]:
# Create strategy and run server
strategy = SaveModelStrategy( 
    # (same arguments as FedAvg here) // https://github.com/adap/flower/blob/main/examples/android/server.py
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=19,
        min_evaluate_clients=19,
        min_available_clients=19,        
        evaluate_fn=get_evaluate_fn(model), # evaluate_fn=None,
        on_fit_config_fn=fit_config,
        on_evaluate_config_fn=evaluate_config,
        initial_parameters=None,
)

9 datasets
0 - PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc
1 - rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw
2 - RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI
3 - VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is
4 - Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw
5 - XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA
6 - YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw
7 - ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM
8 - ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY


In [16]:
# Start Flower server
fl.server.start_server(
    server_address="0.0.0.0:8099",
    config=fl.server.ServerConfig(num_rounds=30),
)

INFO flwr 2023-07-27 14:46:57,401 | app.py:148 | Starting Flower server, config: ServerConfig(num_rounds=30, round_timeout=None)
INFO flwr 2023-07-27 14:46:57,454 | app.py:168 | Flower ECE: gRPC server running (30 rounds), SSL is disabled
INFO flwr 2023-07-27 14:46:57,480 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-27 14:46:57,481 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-27 14:48:32,533 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-27 14:48:32,534 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-07-27 14:48:32,535 | server.py:101 | FL starting
DEBUG flwr 2023-07-27 14:48:32,538 | server.py:218 | fit_round 1: strategy sampled 19 clients (out of 19)
DEBUG flwr 2023-07-27 14:48:58,367 | server.py:232 | fit_round 1 received 19 results and 0 failures
WARNING flwr 2023-07-27 14:48:58,447 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-07-27 14:4

DEBUG flwr 2023-07-27 15:38:48,014 | server.py:168 | evaluate_round 18: strategy sampled 14 clients (out of 14)
DEBUG flwr 2023-07-27 15:39:08,150 | server.py:182 | evaluate_round 18 received 14 results and 0 failures
DEBUG flwr 2023-07-27 15:39:08,151 | server.py:218 | fit_round 19: strategy sampled 14 clients (out of 14)
DEBUG flwr 2023-07-27 15:39:21,849 | server.py:232 | fit_round 19 received 14 results and 0 failures
DEBUG flwr 2023-07-27 15:39:21,857 | server.py:168 | evaluate_round 19: strategy sampled 14 clients (out of 14)
DEBUG flwr 2023-07-27 15:39:47,925 | server.py:182 | evaluate_round 19 received 14 results and 0 failures
DEBUG flwr 2023-07-27 15:39:47,926 | server.py:218 | fit_round 20: strategy sampled 14 clients (out of 14)
DEBUG flwr 2023-07-27 15:40:00,972 | server.py:232 | fit_round 20 received 14 results and 0 failures
DEBUG flwr 2023-07-27 15:40:00,979 | server.py:168 | evaluate_round 20: strategy sampled 14 clients (out of 14)
DEBUG flwr 2023-07-27 15:40:21,332 |

History (loss, distributed):
	round 1: 0.4328664243221283
	round 2: 0.42329931259155273
	round 3: 0.41743314266204834
	round 4: 0.40814149379730225
	round 5: 0.40229853987693787
	round 6: 0.4019489288330078
	round 7: 0.399895578622818
	round 8: 0.40328386425971985
	round 9: 0.4085715115070343
	round 10: 0.40994924306869507
	round 11: 0.41613638401031494
	round 12: 0.4184974730014801
	round 13: 0.4233858287334442
	round 14: 0.42670542001724243
	round 15: 0.4289371371269226
	round 16: 0.4279754161834717
	round 17: 0.4338776469230652
	round 18: 0.4374600052833557
	round 19: 0.4411631226539612
	round 20: 0.4437955617904663
	round 21: 0.4443605840206146
	round 22: 0.44659245014190674
	round 23: 0.4488733112812042
	round 24: 0.4493168592453003
	round 25: 0.4469124972820282
	round 26: 0.4445071816444397
	round 27: 0.44396111369132996
	round 28: 0.44345587491989136
	round 29: 0.4414026737213135
	round 30: 0.4402775168418884